# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [0]:
!pip install tiktoken

     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 780 kB 64.8 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63f16a91-7710-46fc-a4f3-7b3e7ea81428/bin/python -m pip install --upgrade pip' command.


In [0]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

Out[3]: [15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [0]:
import tiktoken

# Obtén el encoding, en este caso usando "p50k_base"
encoding = tiktoken.get_encoding("p50k_base")

# Define una función para mostrar los tokens de una frase
def show_tokens(phrase):
    tokens = encoding.encode(phrase)  # Codifica la frase a tokens
    print(f"Frase: {phrase}")
    print("Tokens:", tokens)
    print("Tokens (como texto):", [encoding.decode([token]) for token in tokens])  # Decodifica los tokens a texto

# Pide al usuario una frase de entrada
input_phrase = input("Introduce una frase: ")

# Muestra los tokens de la frase
show_tokens(input_phrase)


Introduce una frase:  Hola soy Leo, como estas?

Frase: Hola soy Leo, como estas?
Tokens: [39, 5708, 17797, 19632, 11, 401, 78, 1556, 292, 30]
Tokens (como texto): ['H', 'ola', ' soy', ' Leo', ',', ' com', 'o', ' est', 'as', '?']


Let's write a function to count the number of tokens in a text string.

In [0]:
import tiktoken

def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    # Obtener el encoding según el nombre
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Codificar la cadena en tokens
    tokens = encoding.encode(string)
    
    # Retornar el número de tokens
    return len(tokens)

# Ejemplo de uso
num_tokens = get_num_tokens_from_string("Hello World, this is fun!")
print(f"Number of tokens: {num_tokens}")


Number of tokens: 7


## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [0]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [0]:
import re

def normalize_text(string, sep_token=" "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # Convierte todo el texto a minúsculas
    string = string.lower()

    # Reemplaza saltos de línea y secuencias de espacios por un solo espacio
    string = re.sub(r'\s+', sep_token, string).strip()

    # Eliminar puntuación innecesaria, por ejemplo, eliminar comas seguidas de puntos
    string = re.sub(r'[.,]+', ".", string)

    # Eliminar espacios innecesarios alrededor de los puntos
    string = re.sub(r'\s*\.\s*', '.', string)

    # Eliminar saltos de línea (si es que aún quedan)
    string = string.replace("\n", sep_token).strip()

    return string

# Ejemplo de uso
input_text = "Hello World! This is a test.. \n Another line of text.  "
normalized_text = normalize_text(input_text)
print(normalized_text)


hello world! this is a test.another line of text.


## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [0]:
import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    # Obtener la codificación según el nombre proporcionado
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Codificar el texto en tokens
    tokens = encoding.encode(text)
    
    # Si el texto no excede el límite de tokens, devolverlo tal cual
    if len(tokens) <= max_tokens:
        return [text]
    
    # Si excede el límite, dividir el texto en segmentos de tamaño adecuado
    segments = []
    for i in range(0, len(tokens), max_tokens):
        segment_tokens = tokens[i:i + max_tokens]
        segment = encoding.decode(segment_tokens)
        segments.append(segment)
    
    return segments

# Prueba la función
text = "Lorem ipsum " * 500  # Texto largo
segments = validate_text_length(text) # Llamamos la funcion

# Verifica el número de segmentos y su longitud
print(len(segments))  # Resultado esperado: Más de 1 segmento
print(len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))  # <= XXX tokens
##Resultado esperado:

##El texto se divide en segmentos adecuados para ser procesados por OpenAI.
##Cada segmento tendrá un número de tokens menor o igual al límite especificado.


1
1001


## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [0]:
import re

def filter_prohibited_words(text, prohibited_words):
    """Filtra palabras prohibidas del texto y las reemplaza por [REDACTED]."""
    for word in prohibited_words:
        # Usamos una expresión regular para encontrar las palabras completas y reemplazarlas
        text = re.sub(r'\b' + re.escape(word) + r'\b', '[REDACTED]', text, flags=re.IGNORECASE)
    
    return text

# Prueba de la función
text = "This document contains confidential and secret information."
prohibited_words = ['password', 'confidential', 'secret']

cleaned_text = filter_prohibited_words(text, prohibited_words)
print(cleaned_text)  # Resultado esperado: "This document contains [REDACTED] and [REDACTED] information."
#Resultado esperado:
#El texto tendrá palabras prohibidas reemplazadas por [REDACTED].



This document contains [REDACTED] and [REDACTED] information.


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [0]:
!pip install langdetect

     |████████████████████████████████| 981 kB 6.8 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=1e72346daab60ab8adb5df1fc669b799b3c67144a7d23123eb7b4b7127a3d98c
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-63f16a91-7710-46fc-a4f3-7b3e7ea81428/bin/python -m pip install --upgrade pip' command.


In [0]:
from langdetect import detect

def detect_language(text):
    """Detecta el idioma de un texto."""
    return detect(text)

# Prueba la función
text = "Hola, este es un texto en español."
language = detect_language(text)
print(language)  # Resultado esperado: 'es' (para español)
##Resultado esperado:
##El idioma del texto detectado (es, en, etc.) será devuelto.


es


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [0]:
def generate_effective_prompt(topic, audience="general audience"):
    """Genera un prompt optimizado para OpenAI."""
    return f"Explain {topic} in simple terms for a {audience}"

# Prueba la función
topic = "machine learning"
prompt = generate_effective_prompt(topic, audience="beginner")
print(prompt)  # Resultado esperado: "Explain machine learning in simple terms for a beginner."




Explain machine learning in simple terms for a beginner


## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [0]:
def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    """Calcula el costo estimado de un texto basado en el número de tokens."""
    # Obtener la codificación especificada
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Codificar el texto a tokens
    tokens = encoding.encode(text)
    
    # Calcular el número de tokens
    num_tokens = len(tokens)
    
    # Calcular el costo en base al número de tokens
    cost = (num_tokens / 1000) * cost_per_1k_tokens
    
    return cost

# Prueba la función
text = "This is a sample text." * 100
cost = estimate_cost(text)
print(cost)  # Resultado esperado: Costo aproximado basado en la cantidad de tokens



0.01002


## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [0]:
import json
import re

def clean_value(value):
    """Limpia un valor eliminando caracteres no ASCII."""
    if isinstance(value, str):  # Si el valor es una cadena de texto
        return re.sub(r'[^\x00-\x7F]+', '', value)  # Eliminar caracteres no ASCII
    return value  # Si no es una cadena, lo dejamos tal cual

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII de las claves y valores."""
    # Limpiar tanto las claves como los valores
    return {clean_value(k): clean_value(v) for k, v in json_obj.items()}

# Prueba de la función
data = {"key1": "Hello 😊", "key2": "Café and thé"}
cleaned_data = clean_json_text(data)
print(cleaned_data)  # Resultado esperado: {"key1": "Hello ", "key2": "Caf and th"}

##Resultado esperado:
##Un JSON sin caracteres no ASCII.

{'key1': 'Hello ', 'key2': 'Caf and th'}
